<a href="https://colab.research.google.com/github/abirhazra/Reinforcement-Learning/blob/main/Deep_Reinforcement_Learning(gym_environment).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Reference: Nicholas Renotte

#Steps for Deep Reinforcement Learning
1) Creating environments from OpenAi 

2) Building a tf.keras DL model

3) Using keras-rl agents to train

In [4]:
!pip install gym
!pip install keras
!pip install keras-rl2
!pip install tensorflow==2.3.0

     |████████████████████████████████| 61kB 3.8MB/s 


In [5]:
import gym
import random

In [8]:
env = gym.make('CartPole-v0')